# 2.1 Using Tools

By binding tools or functions to your llm, the model gets information about those tools, and it will tell you that it wants to execute a tool. Notice the LLM is not going to execute the tool, its the server where the process is running. That same process will also return the result of those tools back to the LLM.

In [1]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolCall, ToolMessage
from typing import Literal
from langchain.tools import tool

Define the model

In [2]:
llm = AzureChatOpenAI(model="gpt-4.1-mini")

Define the tool/action the LLM will have access to

In [3]:
@tool
def weather_tool(location: Literal["Chicago", "New York", "Los Angeles"]) -> str:
    """
    Get the current weather for a specified location.
    Args:
        location (str): The name of the city to get the weather for. 
                        Must be one of "Chicago", "New York", or "Los Angeles".
    Returns:
        str: A string describing the current weather in the specified location.
    """

    weather_data = {
        "New York": "Sunny, 25°C",
        "Los Angeles": "Cloudy, 22°C",
        "Chicago": "Rainy, 18°C"
    }
    return weather_data.get(location, "Weather data not available for this location.")

Link the tools to the LLM

In [4]:
llm_with_tools = llm.bind_tools([weather_tool])

Set prompt

In [5]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that can provide weather information for specific cities."
    ),
    HumanMessage(
        content="What is the weather like in NYC?"
    )
]

Initial call to LLM

In [6]:
response = llm_with_tools.invoke(input=messages)
#Adding the response to the prompt history
messages.append(response)

Check if the LLM wants to execute a tool call

In [7]:
if len(response.tool_calls) > 0:
    for tool_call in response.tool_calls:
        messages.append(
            # Adding the tool call to the prompt history
            ToolMessage(
                name=tool_call["name"],
                content=weather_tool.run(tool_call["args"]["location"]), #Executing the tool
                tool_call_id=tool_call["id"],
            )
        )

    # Calling the LLM again with the updated messages, containing the tool response
    response = llm_with_tools.invoke(input=messages)
    # Adding the final response to the prompt history
    messages.append(response)

Print result

In [8]:
for msg in messages:
    msg.pretty_print()

================================ System Message ================================

You are a helpful assistant that can provide weather information for specific cities.
================================ Human Message =================================

What is the weather like in NYC?
================================== Ai Message ==================================
Tool Calls:
  weather_tool (call_4d03xCDEnFotRMjeg8w5UP4j)
 Call ID: call_4d03xCDEnFotRMjeg8w5UP4j
  Args:
    location: New York
================================= Tool Message =================================
Name: weather_tool

Sunny, 25°C
================================== Ai Message ==================================

The weather in New York City is currently sunny with a temperature of 25°C.


Notice the Tool Calls and Tool Messages before the response is generated.

**NOTE:** Usage of tool means 2 LLM calls, one to know what tool is necessary and second one containg the result of the tool call